# Tarea 1: Activaciones y pasada hacia adelante en una red neuronal <br/> CC6204 Deep Learning, Universidad de Chile  <br/> Hoja de respuestas
## Nombre: 
Fecha de entrega: 2 de octubre de *2020*

In [1]:
# Este notebook está pensado para correr en CoLaboratory. 
# Lo único imprescindible por importar es torch 
import torch

# Posiblemenete quieras instalar e importar ipdb para debuggear.
# Si es así, descomenta lo siguiente
# !pip install -q ipdb
# import ipdb

# Aqui instalamos la libreria de correccion del curso
!pip install "git+https://github.com/dccuchile/CC6204.git@master#egg=cc6204&subdirectory=autocorrect"

  Cloning https://github.com/dccuchile/CC6204.git (to revision master) to /tmp/pip-install-146g4y0s/cc6204
  Running command git clone -q https://github.com/dccuchile/CC6204.git /tmp/pip-install-146g4y0s/cc6204
  Created wheel for cc6204: filename=cc6204-0.3.2-cp36-none-any.whl size=4708 sha256=9a5f3d55b5eb8833dbd4451c5d8c6fff42566d0a9c82df374e765c0337ad2a8b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ya687nd2/wheels/62/f0/30/aadcb7ce24a2f9c935890518e902d4e23bf97b80f47bb64414
Successfully built cc6204


In [2]:
# importamos las herramientas del curso
from cc6204 import AutoCorrect, FailedTest

# ingresa el host y port que posteamos en u-cursos

corrector = AutoCorrect(host='cc6204.dcc.uchile.cl', port=443)

# anota el token que te daremos en u-cursos

token = ']ye/Ox;nsz'

Connection stablished


# Parte 1: Funciones de activación y función de salida

## 1a) Funciones de activación

In [9]:
# Sigmoid(T)
def sig(T):
  return torch.reciprocal(1 + torch.exp(-1 * T))

# Tanh(T)
def tanh(T):
  E = torch.exp(T)
  e = torch.exp(-1 * T)
  return (E - e) * torch.reciprocal(E + e)

def relu(T):
  T = torch.tensor(T)
  zeros = torch.zeros_like(T)
  return torch.max(T,zeros)

def swish(T, beta):
  T = torch.tensor(T)
  return T * sig(T*beta)

def celu(T, alpha):
  T = torch.tensor(T)
  import math
  def celu_i(x):
    '''
    Aux function
    '''
    if x >= 0:
      return x
    else:
      return alpha*(math.exp(x/alpha)-1)

  return T.apply_(celu_i)

In [10]:
# correccion
test_relu = corrector.get_test_data(homework=1, question="1a", test=1, token=token)
test_swish, swish_par = corrector.get_test_data(homework=1, question="1a", test=2, token=token)
test_celu, celu_par = corrector.get_test_data(homework=1, question="1a", test=3, token=token)


# probablemente quieras convertr los variables test_* a un tensor, ya que por defecto son listas

corrector.sumbit(homework=1, question="1a", test=1, token=token, answer=relu(test_relu))
corrector.sumbit(homework=1, question="1a", test=2, token=token, answer=swish(test_swish, swish_par))
corrector.sumbit(homework=1, question="1a", test=3, token=token, answer=celu(test_celu, celu_par))

Using cached test data
Using cached test data
Using cached test data
Correct Test!
Correct Test!
Correct Test!


## 1b) Softmax

(La demostración puedes entregarla en otro archivo o incluirla directamente acá)

---



In [17]:
# Tu código acá

def softmax(T, dim, estable=True):
  T = torch.tensor(T)
  if estable:
    exp_i = torch.exp(T - torch.max(T))
    softm = torch.tensor(exp_i/torch.sum(exp_i, dim, keepdim=True))
    return softm
  else: 
    exp_i = torch.exp(T)
    softm = torch.tensor(exp_i/torch.sum(exp_i, dim, keepdim=True))
    return softm

In [18]:
test_softmax, _dim = corrector.get_test_data(homework=1, question="1b", test=1, token=token)
corrector.sumbit(homework=1, question="1b", test=1, token=token, answer=softmax(test_softmax, dim=_dim))

Using cached test data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Correct Test!


# Parte 2: Red neuronal y pasada hacia adelante (forward)

## 2a) Clase para red neuronal, 2b) Iterando por parametros, 2d) Pasada hacia adelante

In [19]:
# Tu código debiera continuar así 

class FFNN(torch.nn.Module):
  def __init__(self, F, l_h, l_a, C):
    super(FFNN, self).__init__()
    # capa de entrada
    self.input_layer = torch.nn.Parameter(torch.rand(F))
    # capa oculta
    self.hidden_layers = torch.nn.ParameterList([torch.nn.Parameter(torch.rand(hidden_size)) for hidden_size in l_h])
    # funciones de activacion
    self.activation = l_a
    # capa salida
    self.output_layer = torch.nn.Parameter(torch.rand(C))
  
  def resumen(self):
    # usa self.parameters() o self.named_parameters()
    return self.named_parameters()
  
  def forward(self, x):
    # Usa los parámetros y funciones de activación.
    # El valor de retorno debiera ser y = softmax(capa_de_salida).
    h = x @ self.input_layer
    for i, F in enumerate(self.activacion):
      h = F(h @ self.hidden_layers[i])
    out = softmax(h, C, estable = True)
    return out

In [21]:
#instanciemos una red y veamos el resumen:
red_neuronal = FFNN(300,[50,30],[relu,sig],10)
red_neuronal.resumen

<bound method FFNN.resumen of FFNN(
  (hidden_layers): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 50]
      (1): Parameter containing: [torch.FloatTensor of size 30]
  )
)>

## 2c) Moviendo los parámetros entre dispositivos

In [22]:
# Tu código acá
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
red_neuronal.to(device)

cpu


FFNN(
  (hidden_layers): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 50]
      (1): Parameter containing: [torch.FloatTensor of size 30]
  )
)

# Parte 3: Probando tu red con parámetros pre-entrenados para MNIST

In [23]:
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

# Descarga y almacena el conjunto de prueba de MNIST.
dataset = datasets.MNIST(root='./data',
                             train=False,
                             download=True,
                             transform=ToTensor())
print('Cantidad total de datos:',len(dataset))

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!
Cantidad total de datos: 10000


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 3b) Cargando los parámetros pre-entrenados

In [ ]:
# Tu código acá

## 3c) Calcula la predicción de un ejemplo al azar

In [ ]:
# Tu código acá

## 3d) Pasando todos los ejemplos por la red con un `DataLoader`

In [ ]:
# Acá tu código
def calcula_acierto(red, dataset, batch_size=100, device='cuda'):
  pass

### Corrección red

In [ ]:
from torch.utils.data import Subset
indices = corrector.get_test_data(homework=1, question="red", test=1, token=token)
test_set = Subset(dataset, indices)

In [ ]:
your_network = ...
result = ...
# Montar el `test_set` en un DataLoader y guardar el resultado en un puro tensor

corrector.sumbit(homework=1, question="red", test=1, token=token, answer=result)

## 3e) Opcional: Muestra los casos en donde la red se equivoca

In [ ]:
# Acá tu código

## 3d) Opcional: Crea tus propios ejemplos de dígitos para clasificar

In [ ]:
# Acá tu código